In [1]:
# %% codecell
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
import scipy.spatial
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import timeIntervalPlotter as intervalPlotter
import pysal
import warnings
import lumping_traditional as oldLumping
import boundaryFull_SS_WeightedLumping as WLumping
from importlib import reload
from scipy.stats import rayleigh
import dataManipulation as dataMan
import matplotlib.pyplot as plt
import processData as processData
import tensorflow as tf
warnings.filterwarnings('always')

# <codecell>
#*******************************************************************************
# %%codecell
reload(classImportLA)
reload(processData)
dataFrame = processData.processingData()
#address is: /home/netlab/Desktop/thesis/data/1node1-3-5/
#/home/netlab/Desktop/thesis/data/500f80271400/
#/home/ubuntu/thesis/data/500f80271400/
#/home/sepehr/thesis/data/500f80271400/
data = dataFrame.copy() #copying the dataFrame to have a copy of not edited data

print("hello")
# <codecell>

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/home/sepehr/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/backend.py:5747: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/home/sepehr/.keras/keras.json' mode='r' encoding='UTF-8'>



the address of the collected data files (not alligned files or CSV files): /home/sepehr/thesis/data/500f80271400/
['500f80271400.txt']
here
500f80271400.txt is in csvChecker

we have the csv file: pulling out data

   col1                time  CU
0     0 2018-11-13 02:01:32  45
1     1 2018-11-13 02:01:38  45
2     2 2018-11-13 02:01:44  51
3     3 2018-11-13 02:01:50  53
4     4 2018-11-13 02:01:56  45
now we have the processed data from pandas
hello


In [2]:
# %% codecell
# print(data)
data = processData.dataFrameManipulation(data)

# def diff(x):
#     if (x["col1"] != 0) and (
#         (x["col1"] - 1) in data["col1"]) and (
#             x["timeIndex"] == data["timeIndex"][x["col1"] - 1]) :
#         return data["CU"][x["col1"]] - data["CU"][x["col1"] - 1]
#
#     else:
#         prev_timeIndex = x["timeIndex"]
#         return 0
#
# data["diff"] = data.apply(lambda x: diff(x), axis = 1)
# data["diff"] = data["diff"] + 255

numberOfStates = 255
cuTrans = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans = processData.normalizingTransMatrix(cuTrans)
cuTrans_cpy = normalizedCuTrans.copy()


arrExtra = [0]
print("done")
def particling(x):
    if (x["col1"] - 1 not in data["col1"]) or (x["col1"] - 1 < 0):
        arrExtra[0] = x["time"]
        return 0

    elif x["timeIndex"] != data["timeIndex"][x["col1"] - 1]:
        arrExtra[0] = x["time"]
        return 0

    else:
        return ((x["time"] - arrExtra[0]).seconds)/6

data["periodParticle"] = data.apply(lambda x: particling(x), axis = 1) #numbering each 6 seconds in the dataFrame

print(cuTrans_cpy[0])

please enter how long would be the chunk minutes? 30
removing weekends from the data
done
[[0.34180791 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [3]:
def assymetricDistrib(percentageVector, arrayOfStatesPercentage, predictedStateIndex):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338

    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    minState = np.argmin(inverseDistrib)

    avgUnderUtilizedSum = 0
    numberOfUnderUtilizedStates = minState - 0
    for i in range(predictedStateIndex + 1, len(arrayOfStatesPercentage)):
        underUtilizedPercentage = percentageVector[i][1] - percentageVector[predictedStateIndex][1]
        index_On_UnderUtilized_Distribution = math.floor((underUtilizedPercentage *
                                                          numberOfUnderUtilizedStates) / 100)
        index_On_UnderUtilized_Distribution = minState - index_On_UnderUtilized_Distribution
        avgUnderUtilizedSum += inverseDistrib[index_On_UnderUtilized_Distribution] * arrayOfStatesPercentage[i]

    if predictedStateIndex != len(arrayOfStatesPercentage) - 1:
        avgUnderUtilizedSum /= (len(arrayOfStatesPercentage) - (predictedStateIndex + 1))



    avgOverUtilizedSum = 0
    numberOfOverUtilizedStates = maxState - minState
    for i in range(0, predictedStateIndex):
        overUtilizedPercentage = percentageVector[predictedStateIndex][1] - percentageVector[i][1]
        index_On_OverUtilized_Distribution = math.floor((overUtilizedPercentage *
                                                         numberOfOverUtilizedStates) / 100)
        index_On_OverUtilized_Distribution += minState
        avgOverUtilizedSum += inverseDistrib[index_On_OverUtilized_Distribution] * arrayOfStatesPercentage[i]

    if predictedStateIndex != 0:
        avgOverUtilizedSum /= (predictedStateIndex)

    avgPenalty = avgOverUtilizedSum + avgUnderUtilizedSum

    # avgPenalty *= 1-(arrayOfStatesPercentage[predictedStateIndex])
    return avgPenalty

def statePenaltyValue(transitionMatrix, percentageMatrix):
    transitionMatrix_penalty = np.zeros(shape = (
        transitionMatrix.shape[0], transitionMatrix.shape[0]))

    for row in range(transitionMatrix.shape[0]):
        for col in range(transitionMatrix.shape[0]):
            transitionMatrix_penalty[row][col] = assymetricDistrib(percentageMatrix,
                                                            transitionMatrix[row], col)
        # print(transitionMatrix_penalty[row])

    return transitionMatrix_penalty

def bandwidthPercentage(vectorMatrix):
    percentageIncreament = (100 / vectorMatrix.shape[0])
    percentageMatrix = []
    maxPercentage = 0
    for j in range(vectorMatrix.shape[0]):
        maxPercentage += percentageIncreament
        percentageMatrix.append([[j], maxPercentage, False])

    return percentageMatrix

In [4]:
def preparingMatrixForLumping(transitionMatrix):
    percentageMatrix_list = bandwidthPercentage(transitionMatrix)
    zero_cols_rows = []
    
    #********NORMALIZING************
    for i in range(transitionMatrix.shape[0]):
        sum = np.sum(transitionMatrix[i])
        if sum != 0:
            transitionMatrix[i] = transitionMatrix[i] / sum
    
    #*************removing zeros from columns and rows (matrix reduction) ****************
    for i in range(len(transitionMatrix)):
        if (np.sum(transitionMatrix[i]) == 0) and (np.sum(transitionMatrix[:,i]) == 0):
            zero_cols_rows.append(i)
            
    irreducible_matrix = transitionMatrix.copy()
    for i in range(len(zero_cols_rows) - 1, -1, -1):
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 0)
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 1)
        percentageMatrix_list[zero_cols_rows[i]][2] = True
        
    for i in range(len(percentageMatrix_list) - 1, 0, -1):
        if (percentageMatrix_list[i][2] == True) and (percentageMatrix_list[i - 1][2] == True):
            percentageMatrix_list[i - 1][0].extend(percentageMatrix_list[i][0])
            percentageMatrix_list[i - 1][1] = percentageMatrix_list[i][1]
            del percentageMatrix_list[i]

    for i in range(len(irreducible_matrix)):
        if np.sum(irreducible_matrix[i], dtype = np.float32) != 1.0:
            print(np.sum(irreducible_matrix[i], dtype = np.float32))

    penaltyMatrix = statePenaltyValue(irreducible_matrix, percentageMatrix_list)
    sum = 0
    for i in range(penaltyMatrix.shape[0]):
        sum = np.sum(penaltyMatrix[i])
        if sum != 0:
            penaltyMatrix[i] = penaltyMatrix[i] / sum
    
    return percentageMatrix_list, penaltyMatrix

In [14]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [15]:
for i in [6, 9, 11, 12, 44]:
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    name = "/home/sepehr/thesis/APDataML/pickles/penalty_lumping_result_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(result, handle)

0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  0
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
we have these many sectors to check:  0
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001


KeyboardInterrupt: 

In [16]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(48)]
for timeIndex in range(48):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/penaltyPickles/penalty_lumping_result_" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/penaltyBoundaries_30min.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [18]:
for i in range(48):
    print(i)
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(cuTrans_cpy[i])
    name = "/home/sepehr/thesis/APDataML/pickles/penalty_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0
1.0000001
0.99999994
1.0000001
0.99999994
1
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
2
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
3
0.9999999
0.99999994
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
1.0000001
4
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
5
0.9999999
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
6
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
7
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
8
0.99999994
1.0000001
9
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
10
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
11
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
12
0.99999994
0.99999994
1.0000001
0.99999994
13
0.99999994
0.99999994
0.99999994
14
0.99999994
0.99999994
15
0.99999994
1.0000001